In [24]:
import pandas as pd
import math
import re
import datetime
from pprint import pprint
import json
import uuid
from fuzzywuzzy import fuzz, process
from sqlglot import parse_one, exp
from pprint import pprint
import sys
import os
import shutil







global_dataframe = pd.DataFrame()

# --------------------------------------------------------------------------------------------------------------------------#
# QUERY
# --------------------------------------------------------------------------------------------------------------------------#

file_name = "Life - LFN Pipeline Summary"
# file_name = "Life - Wire Pipeline Reconciliation"

input_path = f"./input/{file_name}.sql"
output_path = f"./OutputQueries/{file_name}/Converted_{file_name}.sql"
with open(input_path, "r") as sql_file:
    sql_string = sql_file.read()

def handle_exception(exception_type, message):
    print(
        f"EXCEPTION: {exception_type.__name__}")
    print(message)
    print("------------------------------------------------------------------------------------------------------------------------")
    sys.exit()
    


class MoreThanOneRowRetrieved(Exception):
    pass

class NoReplacementFound(Exception):
    pass

class NoReplacementFound2(Exception):
    pass


#utils.py
def create_or_overwrite_folder(folder_path):
    try:
        if os.path.exists(folder_path):
            shutil.rmtree(folder_path)
        os.makedirs(folder_path)
    except Exception as e:
        print(f"An error occurred: {e}")


folder_path = f"C:/Users/vbohara/Desktop/Varun/Convtr/OutputQueries/{file_name}"
create_or_overwrite_folder(folder_path)

#utils.py
def append_to_global_dataframe(alias, source_table, source_column, replacement, comment, global_dataframe):
    global_dataframe = pd.concat([global_dataframe, pd.DataFrame({ 
                                        'ALIAS': [alias], 
                                        'SOURCE TABLE': [source_table], 
                                        'SOURCE COLUMN': [source_column], 
                                        'REPLACEMENT': [replacement],
                                        'COMMENT' : [comment]
                                        })], axis=0, ignore_index=True)
    return global_dataframe
#utils.py
def lowercase_listElements(input_list):
    return [x.lower() for x in input_list]


#utlis.py
def list_to_dataframe(column_name, list_name):
    return pd.DataFrame({column_name:[]}) if not list_name else pd.DataFrame({column_name: list_name})

#utils.py
def get_mapping_dataframe(file_path):
    mapping_dataframe = pd.read_excel(
        file_path, sheet_name='Table_Mapping')
    mapping_dataframe = mapping_dataframe.fillna('TBD')
    mapping_dataframe = mapping_dataframe.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    return mapping_dataframe

#utils.py
def get_information_schema_df(file_path):
    information_schema_df = pd.read_excel(
        file_path)
    information_schema_df = information_schema_df.fillna('TBD')
    information_schema_df = information_schema_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    return information_schema_df

#utils.py
def remove_similar_duplicates(input_list, threshold=95):
    unique_items = []
    for item in input_list:
        is_unique = True
        for unique_item in unique_items:
            if fuzz.ratio(item, unique_item) >= threshold:
                is_unique = False
                break
        if is_unique:
            unique_items.append(item)
    return unique_items

#utils.py
def get_best_match(word, word_list, threshold=95):
    best_match = process.extractOne(word.upper(), word_list, scorer=fuzz.ratio)
    return best_match[0] if best_match[1] >= threshold else None 

#utils.py
def extract_from_and_join_clauses(text):
    pattern = r'(?i)(from|join)\s+(\S+)'
    matches = re.findall(pattern, text)
    return matches

#utils.py
def extract_aliases_with_as_from_sql(query):
    pattern = r'as\s+(?:"[^"]*"|\[[^\]]*\]|\w+|\$+\w+\$+|\s*\'[^\'\n]*\'\s*)'
    matches = re.findall(pattern, query, flags=re.IGNORECASE | re.MULTILINE)
    return matches

#utils.py
def extract_where_clauses(query):
    # Split the query into clauses based on common SQL keywords
    clauses = re.split(r'\b(SELECT|FROM|WHERE|GROUP BY|JOIN)\b', query, flags=re.IGNORECASE)
    where_clauses = []
    for i in range(1, len(clauses), 2):
        clause_type = clauses[i].strip().lower()
        if clause_type == 'where':
            where_clauses.append(clauses[i + 1])
    return where_clauses

#utils.py
def extract_and_or_of_where_clause(where_clause):
    sections = re.split(r'\b(?:and|or)\b', where_clause)
    sections = [section.strip() for section in sections]
    return sections

#utils.py
def extract_on_clause_expressions(query):
    # FROM THE EXTRACTED ON CLAUSES WE ARE IGNORING THOSE WITH CASE STATEMENTS
    clauses = re.split(r'\b(SELECT|FROM|WHERE|GROUP BY|ON|JOIN|CASE|WHEN|ELSE|END)\b', query, flags=re.IGNORECASE)
    on_clauses = []
    for i in range(1, len(clauses), 2):
        clause_type = clauses[i].strip().lower()
        if clause_type == 'on':
            on_clauses.append(clauses[i + 1])
    # ONLY CHOSSING THOSE THAT HAVE = IN THEM 
    return [clause.strip() for clause in on_clauses if '=' in clause]

#utils.py
def preety_print_dict(dictionary):
    dictionary = json.dumps(dictionary, indent=4)
    print(dictionary)

#utils.py     
def tempoary_replace_alias_in_output_query(input_query, output_query):
    #--------------------------------------------------------------------------------------------------------------------------#
    alias_to_code_list = extract_aliases_with_as_from_sql(input_query)
    #  Stripping leading and trailing whitespace, and removing any non-word and non-whitespace characters from each string.
    for i in range(len(alias_to_code_list)):
        alias_to_code_list[i] = re.sub(r'[^\w\s]', '', alias_to_code_list[i].strip())

    #Generation of the code
    alias_to_code = {item: f" {str(uuid.uuid4())[:8]} " for item in alias_to_code_list}
    code_to_alias = {code: item for item, code in alias_to_code.items()}
    #--------------------------------------------------------------------------------------------------------------------------#
    #                                   OP. TEMPOARY REPLACE ALIAS IN OUTPUT QUERY 
    #--------------------------------------------------------------------------------------------------------------------------#
    for key in alias_to_code:
        output_query = output_query.replace(key, alias_to_code[key])
                                                                #END OF ALIAS
    #--------------------------------------------------------------------------------------------------------------------------#
    return output_query, code_to_alias

#utlis.py
def dict_to_dataframe(input_dict,column1,column2):
    data = {column1: list(input_dict.keys()), column2: [', '.join(value) for value in input_dict.values()]}
    df = pd.DataFrame(data)
    return df

#utils.py
def single_dict_to_dataframe(input_dict,column1,column2):
    data = {column1 : list(input_dict.keys()), column2: list(input_dict.values())}
    df = pd.DataFrame(data)
    return df

#table_utils.py
def table_mapping(df,tables_in_sttm):
    mapping_dict = {}
    df = df[df['DW Table'].isin(tables_in_sttm)]
    for index, row in df.iterrows():
        source_table = row["DW Table"].lower()
        target_table = row["Physical Table Name"]
        mapping_dict[source_table] = target_table
    return mapping_dict

#table_utils.py
def replace_table_name(query, table_name):
    pattern = r'(\w+)\.' + re.escape(table_name) + r'\b'
    modified_query = re.sub(pattern, r'table_name', query)
    return modified_query

#tables_utils.py
def generate_cte_statements_for_tables(df, table_list,data_type_defaults):
    tables_not_to_be_considered_for_metadata = []
    if not table_list:
        return ''
    cte_statements = {}
    for table_name in table_list:
        table_df = df[df['TABLE_NAME'] == table_name]
        if table_df.empty:
            print(f"CTE NOT GENERATED FOR {table_name}")
            # table_list.remove(table_name)
            tables_not_to_be_considered_for_metadata.append(table_name)
            if not table_list:
                return ''
            continue
        cte = f"{table_name} as (\n"
        select_clause = f"select "
        for index, row in table_df.iterrows():
            column_name = row['COLUMN_NAME']
            data_type = row['DATA_TYPE'].lower()
            default_value = data_type_defaults.get(data_type)
            select_clause += f"{default_value} as {column_name}, "
        
        select_clause = select_clause.rstrip(', ') 
        cte += select_clause + "\n),\n"
        cte_statements[table_name] = cte
    
    cte_query = "with "+"\n".join(cte_statements.values())
    return cte_query, tables_not_to_be_considered_for_metadata

#table_utils.py
def find_table_aliases(query, table_names):
    # Given a list of tables return a list of tuple (table_name,alias)
    pattern = r'(\b{}\b)\s+AS\s+(\w+)'.format('|'.join(table_names))
    matches = re.findall(pattern, query, re.IGNORECASE)
    matches = [(item[0].lower(), item[1]) for item in matches]
    return matches

#table_utils.py
def create_alias_table_dict(table_alias_pairs):
    # Given a list of tuple from above function -> {alias:[List of tables with that alias]}
    alias_table_dict = {}
    for table, alias in table_alias_pairs:
        if alias not in alias_table_dict:
            alias_table_dict[alias] = {table}
        else:
            alias_table_dict[alias].add(table)
    return alias_table_dict
    
data_type_defaults ={
                        'bigint' : 1,   'char':"''",
                        'date':f"date('01-01-1900')",
                        'datetime':"date('01-01-1900')",
                        'datetime2':"date('01-01-1900')",
                        'decimal':1.0,  'float':1.0,
                        'int':1,'money':1,'numeric':1,'nvarchar':"''",'smallint':1,
                        'tinyint':1,'varbinary':1,'varchar':"''"
                    }

#table_utils.py
# TABLES IN QUERY = TABLES IN STTM(REPRESENTED WITH THE TABLE NAME FROM STTM) + TABLES NOT IN STTM(REPRESENTED AS NONE)
#--------------------------------------------------------------------------------------------------------------------------#
def extract_table_from_query(input_query, mapping_dataframe):
    tables_in_query_tuple = extract_from_and_join_clauses(input_query)
    tables_in_query_raw = [item[1] for item in tables_in_query_tuple]
    tables_in_query_raw = [item for item in tables_in_query_raw if 'select' not in item.lower()]
    unique_tables_in_query_raw = remove_similar_duplicates(tables_in_query_raw)
    unique_tables_in_query_raw = [item.split('.')[-1] if '.' in item else item for item in unique_tables_in_query_raw]
    translation_table = str.maketrans('', '', '()";')
    unique_tables_in_query_raw = [item.translate(translation_table) for item in unique_tables_in_query_raw]
    sttm_tables = mapping_dataframe['DW Table'].unique().tolist()
    sttm_tables = [item for item in sttm_tables if item!='TBD']
    # TABLES IN STTM  = TABLES IN STTM WITH TARGET TABLES  + TABLES IN STTM WITHOUT TARGET TABLES 
    #--------------------------------------------------------------------------------------------------------------------------#
    tables_in_query = [get_best_match(item, sttm_tables, threshold=95)
                    for item in unique_tables_in_query_raw]
    original_to_bestMatch = dict()
    for original, best_match in zip(unique_tables_in_query_raw, tables_in_query):
        original_to_bestMatch[original] = best_match
    tables_not_in_sttm = [key for key in original_to_bestMatch.keys() if original_to_bestMatch[key] is  None]
    tables_in_sttm = [value.lower() for value in original_to_bestMatch.values() if value is not None]
    return tables_in_sttm,tables_not_in_sttm
  
#column_utils.py
def extract_columns(tsql_query):
    parsed_query = parse_one(tsql_query)
    column_references = parsed_query.find_all(exp.Column)
    column_names = [column.name for column in column_references]
    return list(set(column_names))
    
#column_utils.py
def column_mapping(df, information_schema_df, tables_in_sttm, tsql_query):
    """
    Returns 
           Root Column names that are present in the query 
           Columns that do not belong to any tables that are present in the STTM
    """

    #Extracting only those columns that are present in the STTM
    df = df[df['DW Table'].isin(tables_in_sttm)]
    information_schema_df = information_schema_df[information_schema_df['TABLE_NAME'].isin(tables_in_sttm)]
    columns_in_query = []
    columns_in_query = extract_columns(tsql_query)
    #Making sure that all the columns for the tables that are mentioned in the query , are fetched
    # for column in information_schema_df['COLUMN_NAME']: #Only those columns whose tables MIGHT BE mentioned in the query 
    #     pattern = r'\b' + re.escape(column) + r'\b'
    #     if re.search(pattern, tsql_query): #and (column not in columns_in_query):
    #         columns_in_query.append(column)
    #Columns in sttm whose table is present in sttm
    columns_in_sttm = df['DW Column'].tolist()
    #Columns in query but not in STTM 
    columns_not_in_sttm = list(set(columns_in_query) - set(columns_in_sttm))
    col_datatype_info_schma = information_schema_df[information_schema_df['COLUMN_NAME'].isin(columns_not_in_sttm)]
    col_datatype_info_schma = col_datatype_info_schma.drop_duplicates(subset=["COLUMN_NAME", "DATA_TYPE"])
    columns_not_in_sttm = dict(zip(col_datatype_info_schma["COLUMN_NAME"], col_datatype_info_schma["DATA_TYPE"]))
    return list(set(lowercase_listElements(columns_in_query))) , columns_not_in_sttm


#column_utils.py
def get_replcmnt_unique_child_instances(mapping_dataframe, 
                                        alias_table_dict, 
                                        child_instance_alias, 
                                        child_instance_root,
                                        data_type_defaults,
                                        information_schema_df):
    try:
        global global_dataframe
        #Unique Alias-> Has only 1 Table in it's set 
        # THIS CONDITION SHOULD RETURN ONLY 1 ROW
        filtered_df = mapping_dataframe[
            (mapping_dataframe['DW Table'] == (list(alias_table_dict[child_instance_alias])[0].lower())) &
            (mapping_dataframe['DW Column'] == child_instance_root.lower()) &
            (mapping_dataframe['Physical Column Name'] !='tbd')
            ]
        if not filtered_df.empty:
            if filtered_df.shape == (1, filtered_df.shape[1]):
                global_dataframe = append_to_global_dataframe(child_instance_alias.lower(),
                                                              ', '.join(
                                                                  lowercase_listElements(filtered_df['DW Table'].tolist())),
                                                              ', '.join(
                                                                  lowercase_listElements(filtered_df['DW Column'].tolist())),
                                                            filtered_df['Physical Column Name'].iloc[0], 
                                                            "Unique Alias : Replacement found in STTM",
                                                            global_dataframe)
                return filtered_df['Physical Column Name'].iloc[0]
            else: 
                raise MoreThanOneRowRetrieved("MORE THAN 1 ROW RETRIEVED")
        else:
            # This case infers that source/target column is missing (TBD) (THE 2nd or 3rd CONDITION FAILED)
            # Because the values of Table and column will always be present
            global_dataframe = append_to_global_dataframe(child_instance_alias.lower(),
                                                            list(alias_table_dict[child_instance_alias])[0].lower(), 
                                                            child_instance_root.lower(), 
                                                            f"/*{child_instance_root} */", 
                                                              f"Commented.",
                                                            global_dataframe)
            return f"/*{child_instance_root} */"
        
    
    except MoreThanOneRowRetrieved as e:
        return handle_exception(MoreThanOneRowRetrieved,f"CHECK STTM\n{child_instance_alias} : {list(alias_table_dict[child_instance_alias])[0]} : {child_instance_root}\n{filtered_df}")
    except NoReplacementFound as e:
        return handle_exception(NoReplacementFound,f"THIS CASE SHOULD NEVER OCCUR, SOMETHING WRONG WITH THE INFORMATION SCHEMA\n{child_instance_alias} : {list(alias_table_dict[child_instance_alias])[0]} : {child_instance_root}")
    except Exception as e:
        return handle_exception(Exception,str(e))
    

#column_utils.py
def get_child_instances(root_col,query):
    """
    INPUT : GIVEN A ROOT COLUMN AND THE QUERY, 
    OUTPUT: A LIST OF ALL THE INSTANCES OF THE ROOT COLUMN 
            METADATA ON HOW MAY STAND ALONE INSTNACES 
    """
    pattern = r'\b\w*\.' + re.escape(root_col) + r'\b|\b' + re.escape(root_col) + r'\b'
    matches = re.findall(pattern, query, re.IGNORECASE)
    set_column_instances = list(set(matches))
    #Focus on the columns with '.'
    column_instance_list = [item for item in set_column_instances if '.' in item]
    number_of_standalone_root_col = len([item for item in set_column_instances if '.' not in item])
    root_col_meta_data ={'number_of_standalone_root_col':number_of_standalone_root_col}
    return list(set(lowercase_listElements(column_instance_list))), root_col_meta_data

#column_utils.py
def get_replcmnt_duplicate_child_instances(mapping_dataframe,
                                            alias_table_dict,
                                              child_instance_alias,
                                                child_instance_root,
                                                data_type_defaults,
                                                information_schema_df):
    try:
        global global_dataframe
        # Filter the DataFrame based on conditions
        filtered_df = mapping_dataframe[
            (mapping_dataframe['DW Table'].isin(lowercase_listElements(alias_table_dict[child_instance_alias]))) &
        (mapping_dataframe['DW Column'] == child_instance_root.lower()) &
        (mapping_dataframe['Physical Column Name'] != 'tbd')
        ]
        if not filtered_df.empty:
            global_dataframe = append_to_global_dataframe(child_instance_alias.lower(),
                                                          ', '.join(lowercase_listElements(
                                                              alias_table_dict[child_instance_alias])),
                                                            child_instance_root.lower(),
                                                          ', '.join(lowercase_listElements(list(
                                                              filtered_df['Physical Column Name']))),
                                                            f"Duplicate alias : Found replacement in STTM",
                                                            global_dataframe)
            return list(filtered_df['Physical Column Name'])
        else:           
            global_dataframe = append_to_global_dataframe(child_instance_alias,
                                                            ', '.join(alias_table_dict[child_instance_alias]), 
                                                            child_instance_root, 
                                                            f"/*{child_instance_root}*/", 
                                                            f"Commented..",
                                                            global_dataframe)
            return f"/*{child_instance_root}/*"
    except NoReplacementFound2 as e:
        return handle_exception(NoReplacementFound2,
                                f"""EXCEPTION in get_replcmnt_duplicate_child_instances()
                                THIS CASE SHOULD NEVER OCCUR, SOMETHING WRONG WITH THE INFORMATION SCHEMA
                                {child_instance_alias} : {alias_table_dict[child_instance_alias]} : {child_instance_root}""")


#column_utils.py
#IT SHOULD BE COLUMNS_IN_QUERY AND NOT COLUMNS_IN_STTM
def extract_all_column_instance_replacements(columns_in_sttm,query,alias_table_dict,duplicate_alias_to_tables,mapping_dataframe,information_schema_df):
    global file_name
    # Given a set of all the root column names mentioned in the query 
    # Return a dictionary having (child column : replacements)
    metadata_child_instances_to_replacements = dict()
    child_instances_to_replace = dict()

    root_columns_in_sttm = columns_in_sttm
    pprint(list_to_dataframe('ROOT COLUMNS IN STTM',root_columns_in_sttm))
    standalone_root_columns = {}
    for root_column in root_columns_in_sttm:
        # GETTING THE CHILD INSTANCES OF THE ROOT COLUMN OF CURRENT ITERATION
        child_instances, root_col_meta_data = get_child_instances(root_column,query)
        if not child_instances:
            #These columns might have come from information_schema but haven't been used in the query 
            if root_col_meta_data['number_of_standalone_root_col'] == 0 :
                continue
            else:
                # This means that the root column is not mentioned in the query using `.root_column` , it's directly mentioned `root_column`
                # CONCATENATE RHS WITH IDENTIFIER TO CHECK WHICH COLUMNS ARE FROM THIS LOC
                standalone_root_columns[root_column] = str(root_col_meta_data['number_of_standalone_root_col'])
        else:
            metadata_child_instances_to_replacements[root_column] = {}
            # standalone_root_columns[root_column] = str(root_col_meta_data['number_of_standalone_root_col']) # UNCOMMENT IF WANT TO CHECK 0 VALUES 
            if root_col_meta_data['number_of_standalone_root_col'] !=0 :
                standalone_root_columns[root_column] = str(root_col_meta_data['number_of_standalone_root_col'])  
            for child_instance in child_instances:
                child_instance_alias,child_instance_root = child_instance.split('.')[0],child_instance.split('.')[-1]
                #Child instance with only one possible replacement , alias is not duplicated
                if child_instance_alias not in duplicate_alias_to_tables.keys():
                    # print(child_instance_alias)
                    # Ignoring those aliases that belong to tables_not_in_sttm, alias_table_dict has only those tables that are present in STTM
                    if child_instance_alias in alias_table_dict.keys():
                        replacement =  get_replcmnt_unique_child_instances(mapping_dataframe, 
                                                                            alias_table_dict, 
                                                                            child_instance_alias, 
                                                                            child_instance_root,
                                                                            data_type_defaults,
                                                                            information_schema_df)
                        if '/*' in (replacement): # SOURCE/TARGET COLUMN MISSING 
                            metadata_child_instances_to_replacements[root_column][child_instance]=[replacement]
                            child_instances_to_replace[child_instance] = replacement
                        else:
                            metadata_child_instances_to_replacements[root_column][child_instance]=[child_instance.replace(child_instance_root,replacement)]
                            child_instances_to_replace[child_instance] = child_instance.replace(child_instance_root,replacement)
                            
                    else:
                        global global_dataframe
                        #This alias is of that table which is not present in STTM , and there by has a CTE created for it 
                        #This tell us that we are not replacing those columns as we are creating a CTE
                        metadata_child_instances_to_replacements[root_column][child_instance]=[child_instance] #KEEPING IT AS IT IS
                        child_instances_to_replace[child_instance] = child_instance
                        global_dataframe = append_to_global_dataframe(child_instance_alias,
                                                          child_instance_alias, 
                                                          child_instance_root, 
                                                          child_instance, 
                                                          f"CTE: {child_instance_alias} is an alias of table that is not present in the STTM",
                                                          global_dataframe)

                #Child instance with more than one possible replacement , alias is duplicated
                else:
                    replacement_list = get_replcmnt_duplicate_child_instances(mapping_dataframe, 
                                                                            alias_table_dict, 
                                                                            child_instance_alias, 
                                                                            child_instance_root,
                                                                            data_type_defaults,
                                                                            information_schema_df
                                                                            )
                    if isinstance(replacement_list,str): # SOURCE/TARGET COLUMN MISSING 
                        metadata_child_instances_to_replacements[root_column][child_instance]= replacement_list
                        child_instances_to_replace[child_instance] = replacement_list 
                    else:
                        replacement_list = [ child_instance.replace(child_instance_root,item) for item in replacement_list]
                        metadata_child_instances_to_replacements[root_column][child_instance]= replacement_list
                        child_instances_to_replace[child_instance] = str(replacement_list[0]) 
    with open("C:/Users/vbohara/Desktop/Varun/Convtr/metadata_child_instances_to_replacements.json", 'w') as json_file:
        json.dump(metadata_child_instances_to_replacements, json_file, indent=4)
    pprint(dict_to_dataframe(standalone_root_columns,"ROOT COLUMN" , "STANDALONE ROOT COLUMNS"))
    global_dataframe = global_dataframe.sort_values(by=['ALIAS', 'SOURCE COLUMN'])
    global_dataframe.to_excel(f"./OutputQueries/{file_name}/Mapping_{file_name}.xlsx", index=False)
    return child_instances_to_replace

In [25]:
input_query = sql_string.lower() # DID THIS TO RESOLVE THE ISSUE OF CASE SENSITIVITY, ESPECIALLY IN UPPER CASE ALIAS 
input_query_where_clauses_list = extract_where_clauses(input_query)
input_query_on_clause_list = extract_on_clause_expressions(input_query)
output_query = input_query

#STEP_1 : Replace the extra spaces after AS
print("STEP_1 : Replace the extra spaces after AS")
input_query = re.sub(r'\bas\s+', 'as ', input_query)
output_query = re.sub(r'\bas\s+', 'as ', output_query)

#--------------------------------------------------------DATAFRAME---------------------------------------------------------# 
#--------------------------------------------------------------------------------------------------------------------------#
file_path = "C:/Users/vbohara/Desktop/Varun/Convtr/dataset/sttm_new.xlsx"
mapping_dataframe = get_mapping_dataframe(file_path)
file_path = "C:/Users/vbohara/Desktop/Varun/Convtr/dataset/INFORMATION_SCHEMA.xlsx"
information_schema_df = get_information_schema_df(file_path)
#--------------------------------------------------------------------------------------------------------------------------#

#STEP_2 : Extract tables_in_sttm and tables_not_in_sttm
#--------------------------------------------------------------------------------------------------------------------------#
tables_in_sttm,tables_not_in_sttm = extract_table_from_query(input_query, mapping_dataframe)
print("STEP_2 : EXTRACT TABLES IN STTM AND TABLES NOT IN STTM\n")
print("TABLES IN STTM")
pprint(list_to_dataframe("TABLES IN STTM",tables_in_sttm))
print("\nTABLES NOT IN STTM")
pprint(list_to_dataframe("TABLES NOT IN STTM",tables_not_in_sttm))

#STEP_3 : Extract Alias to Table Mapping and Identify duplicate alias if any 
#--------------------------------------------------------------------------------------------------------------------------#
tables_in_query = tables_in_sttm #TBD + tables_not_in_sttm
table_aliases = find_table_aliases(input_query, tables_in_query)
alias_table_dict = create_alias_table_dict(table_aliases)
duplicate_alias_to_tables = dict()
print("\nSTEP_3 : Extract Alias to Table Mapping and Identify duplicate alias if any".upper())
print(f"Alias to Table Mapping\n")
pprint(dict_to_dataframe(alias_table_dict,"ALIAS","TABLE NAMES"))
for alias, tables in alias_table_dict.items():
    if len(tables)>1:
        duplicate_alias_to_tables[alias] = tables

if list(duplicate_alias_to_tables.keys()):
    print(f"\nDuplicate Alias in the queries\t{list(duplicate_alias_to_tables.keys())}\n")
    pprint(dict_to_dataframe(duplicate_alias_to_tables,"ALIAS","TABLE NAMES"))
else:
    print("\nNO DUPLICATE ALIAS FOUND IN THE QUERY!!") 


#STEP_4 : GET THE TABLE MAPPING
table_mapping_dict = table_mapping(mapping_dataframe,tables_in_sttm)
print("\nSTEP_4 : GET THE TABLE MAPPING")
pprint(single_dict_to_dataframe(table_mapping_dict,"SOURCE TABLE","TARGET TABLE"))
columns_in_sttm, columns_not_in_sttm = column_mapping(mapping_dataframe,
                                                      information_schema_df,
                                                      tables_in_sttm,
                                                      input_query)

#STEP_5 : TEMPOARY REPLACE ALIAS IN OUTPUT QUERY
print("\nSTEP_5 : TEMPOARY REPLACE ALIAS IN OUTPUT QUERY")
output_query, code_to_alias = tempoary_replace_alias_in_output_query(input_query, output_query)

#--------------------------------------------------------------------------------------------------------------------------#
#STEP_6 : GET REPLACEMENTS FOR COLUMNS  
child_instances_to_replace =  extract_all_column_instance_replacements(columns_in_sttm,
                                                                            output_query,
                                                                            alias_table_dict,
                                                                            duplicate_alias_to_tables,
                                                                            mapping_dataframe,
                                                                            information_schema_df)


#--------------------------------------------------------------------------------------------------------------------------#
#STEP_7 : REPLACE TABLES
table_dict = dict()
for key in table_mapping_dict.keys():
    if key in output_query:
        #EDGECASE: TABLES IN STTM WITHOUT TARGET TABLES 
        if "tbd" in table_mapping_dict[key]:
            # Send it for "with clause" operation
            tables_not_in_sttm.append(key)
            continue            
        table_dict[key] = f"{table_mapping_dict[key]}"

        # Replacing the table names
        output_query = output_query.replace(key,f"{table_mapping_dict[key]}")
#--------------------------------------------------------------------------------------------------------------------------#
#STEP_8 : REPLACE COLUMNS IN STTM
#--------------------------------------------------------------------------------------------------------------------------#
sorted_keys = sorted(child_instances_to_replace.keys(), key=len, reverse=True)
for key in sorted_keys:            
    pattern = r'\b' + re.escape(key) + r'\b'
    output_query = re.sub(pattern, f"{child_instances_to_replace[key]}", output_query, flags=re.IGNORECASE)
#--------------------------------------------------------------------------------------------------------------------------#
            
cte_statements, tables_not_to_be_considered_for_metadata =  generate_cte_statements_for_tables(information_schema_df,
                                                                                               tables_not_in_sttm,
                                                                                               data_type_defaults)

header_meta_data = f"""
Script Generated By : Converter.py
Date                : {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
Description         : "Converted input.sql to redshift_output.sql"
"""

for key in code_to_alias:
    if key in output_query:
        output_query = output_query.replace(key, code_to_alias[key])


if 'with' in output_query and 'with' in cte_statements:
    output_query = output_query.replace('with', '')
    if cte_statements != '':
        cte_statements = cte_statements +','

output_content = (
    f"/*{header_meta_data}*/\n"
    f"{cte_statements}\n\n"
    f"{output_query}\n"
)

with open(f"C:/Users/vbohara/Desktop/Varun/Convtr/OutputQueries/{file_name}/{file_name}_columnMapping.json", 'w') as json_file:
    json.dump({key: child_instances_to_replace[key] for key in sorted(child_instances_to_replace)} , json_file, indent=4)

with open(f"C:/Users/vbohara/Desktop/Varun/Convtr/OutputQueries/{file_name}/{file_name}_tableMapping.json", 'w') as json_file:
    json.dump(table_mapping_dict, json_file, indent=4)

with open(output_path, 'w') as file:
    file.write(output_content)

STEP_1 : Replace the extra spaces after AS
STEP_2 : EXTRACT TABLES IN STTM AND TABLES NOT IN STTM

TABLES IN STTM
               TABLES IN STTM
0  rpt_dim_is_policy_pend_ext
1        fct_daily_trans_smry
2             rpt_dim_channel
3             rpt_dim_product
4     ref_is_pend_status_code
5              rpt_dim_policy
6                rpt_dim_firm
7             rpt_dim_advisor
8              rpt_dim_agency

TABLES NOT IN STTM
                        TABLES NOT IN STTM
0                rpt_dim_advisor_hierarchy
1  #t2_life_lfn_pipeline_summary_procedure

STEP_3 : EXTRACT ALIAS TO TABLE MAPPING AND IDENTIFY DUPLICATE ALIAS IF ANY
Alias to Table Mapping

  ALIAS                 TABLE NAMES
0   ppe  rpt_dim_is_policy_pend_ext
1     f        fct_daily_trans_smry
2   rdc             rpt_dim_channel
3    pr             rpt_dim_product
4   psc     ref_is_pend_status_code
5   rdp              rpt_dim_policy
6    df                rpt_dim_firm
7   rsm             rpt_dim_advisor
8   rda     